In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/devanagari-character-dataset/labels.csv")

In [ ]:
df.dropna(axis = 0,inplace = True)

In [ ]:
df.rename(
columns = {"Numerals":"Class","Unnamed: 1":"Label","Unnamed: 2":"Devanagari Label","Unnamed: 3":"Phonetics"},inplace = True)

In [ ]:
df.info()

In [ ]:
df.drop_duplicates(inplace = True)

In [ ]:
df.drop(labels = 1,axis = 0,inplace = True)
df.reset_index(inplace = True)

In [ ]:
df

In [ ]:
df.drop(labels = 'index',axis = 1,inplace = True)

In [ ]:
df['type']=0
df['type'].loc[0:9]='numerals'
df['type'].loc[10:21]='vowels'
df['type'].loc[22:]='consonants'

In [ ]:
df

In [ ]:
filenames_number = []
keys = []
dictionary = {}
for i in range(0,10):
    keys.append(i)
    filenames_number.append(os.listdir(f'/kaggle/input/devanagari-character-dataset/nhcd/nhcd/numerals/{i}'))
for keys, filenames_number in zip(keys,filenames_number):
    dictionary[keys] = filenames_number
df_new=pd.Series(dictionary)
table = pd.DataFrame()
new = []
a = 0

for i in range(0,10):
    
    for j in range(0,len(df_new)):
        table['Class']=a
        table['images']=df_new[j]
    new.append(table.copy())
    a = a+1    
    
n_images = pd.DataFrame()
for i in range(0,len(new)):
    n_images=pd.concat(new.copy())     
    
n_images['type']='numerals'    

In [ ]:
filenames_vowels = []
keys = []
dictionary = {}
for i in range(1,13):
    keys.append(i)
    filenames_vowels.append(os.listdir(f'/kaggle/input/devanagari-character-dataset/nhcd/nhcd/vowels/{i}'))
for keys, filenames_vowels in zip(keys,filenames_vowels):
    dictionary[keys] = filenames_vowels
df_new=pd.Series(dictionary)
table = pd.DataFrame()
new = []
a = 0

for i in range(1,14):
    
    for j in range(1,len(df_new)):
        table['Class']=a
        table['images']=df_new[j]
    new.append(table.copy())
    a = a+1    
    
v_images = pd.DataFrame()
for i in range(0,len(new)):
    v_images=pd.concat(new.copy())     
    
v_images['type']='vowels'    

In [ ]:
filenames_consonants = []
keys = []
dictionary = {}
for i in range(1,37):
    keys.append(i)
    filenames_consonants.append(os.listdir(f'/kaggle/input/devanagari-character-dataset/nhcd/nhcd/consonants/{i}'))
for keys, filenames_consonants in zip(keys,filenames_consonants):
    dictionary[keys] = filenames_consonants
df_new=pd.Series(dictionary)
table = pd.DataFrame()
new = []
a = 0

for i in range(1,38):
    
    for j in range(1,len(df_new)+1):
        table['Class']=a
        table['images']=df_new[j]
    new.append(table.copy())
    a = a+1    
    
c_images = pd.DataFrame()
for i in range(0,len(new)):
    c_images=pd.concat(new.copy())     
    
c_images['type']='consonants'    

In [ ]:
frames = [n_images,v_images,c_images]
images = pd.concat(frames,axis =0)
images.reset_index(inplace = True)

In [ ]:
type(df)

In [ ]:
df['Class']=df['Class'].astype(str).astype(int)

In [ ]:
df_images = pd.merge(df,images,on=['Class','type'])

In [ ]:
df_images.drop(labels = 'index',axis =1,inplace = True)

In [ ]:
df_images.reset_index(inplace = True)

In [ ]:
Dev_list = df['Phonetics'].unique()
Dev_list.sort()

In [ ]:
Dev_list

In [ ]:
df_images

In [ ]:
DATASET_PATH = '/kaggle/input/devanagari-character-dataset/nhcd/nhcd'

In [ ]:
import cv2
from tqdm import tqdm

In [ ]:

def GetData(start,stop):
    x_data=[]
    y_data=[]
    
    for i in tqdm(range(start,stop)):
        Image_path = DATASET_PATH + '/'+df_images.loc[i,'type']+'/'+str(df_images.loc[i,'Class'])+'/'+df_images.loc[i,'images']
        image = cv2.imread(Image_path,cv2.IMREAD_GRAYSCALE)

        try:
            
            resized_img = cv2.resize(image,dsize = (28,28))
        except:
            print(i,"Rejected Image",Image_path)
            continue
        
        x_data.append(resized_img)
       
        
        y_values = df_images['Phonetics'][i]
        y_data.append(y_values)
    return x_data,y_data
        

In [ ]:
X = []
Y= []
start = 0
stop = len(df_images)
X,Y = GetData(start,stop)

In [ ]:
from sklearn.preprocessing import LabelEncoder
label = LabelEncoder()
Y =label.fit_transform(Y)

In [ ]:
plt.imshow(X[5000])


In [ ]:
X = np.array(X).reshape(-1,28,28,1)
Y = np.array(Y)
X = X/255.0 
#Y = Y.reshape(len(X),)

In [ ]:
print('Image Data Shape: ',X.shape)
print('Label Data Shape: ',Y.shape)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.1, random_state = 500)

In [ ]:
print('Train Image Data Shape: ',X_train.shape)
print('Train Label Data Shape: ',Y_train.shape)
print('Test Image Data Shape: ',X_test.shape)
print('Test Label Data Shape: ',Y_test.shape)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import os
import tensorflow as tf
import datetime, os
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
label_count = len(df_images['Devanagari Label'].unique())

In [ ]:
def build_model():
    cnn_model = Sequential()
    cnn_model.add(Conv2D(32, (3, 3), input_shape = (28,28,1), activation='relu'))
    cnn_model.add(MaxPooling2D(pool_size = (2, 2)))
    cnn_model.add(Dropout(0.25))

    cnn_model.add(Conv2D(64, (3, 3), input_shape = (28,28,1), activation='relu'))
    cnn_model.add(MaxPooling2D(pool_size = (2, 2)))

    cnn_model.add(Conv2D(128, (3, 3), input_shape = (28,28,1), activation='relu'))
    cnn_model.add(MaxPooling2D(pool_size = (2, 2)))
    

    cnn_model.add(Flatten())
    cnn_model.add(Dense(units = 512, activation = 'relu'))
    cnn_model.add(Dense(units = label_count, activation = 'softmax'))
    return cnn_model


In [ ]:
model=build_model()

In [ ]:
print(model.summary())

In [ ]:
def train_model(model):
    model.compile(loss ='sparse_categorical_crossentropy', optimizer='adam' ,metrics =['accuracy'])
    history = model.fit(X_train, Y_train, batch_size = 128, epochs = 150, verbose = 1, validation_data = (X_test, Y_test))
#     model.fit(X_train, Y_train, epochs=50, batch_size=512, validation_split=0.2)

In [ ]:
train_model(model)

In [ ]:
plt.subplot(2, 2, 1)
plt.plot(model.history.history['loss'], label='Loss')
# plt.plot(model.history.history['val_loss'], label='val_Loss')
plt.legend()
plt.grid()
plt.title('Loss evolution')

plt.subplot(2, 2, 2)
plt.plot(model.history.history['accuracy'], label='accuracy')
plt.plot(model.history.history['val_accuracy'], label='val_accuracy')
plt.legend()
plt.grid()
plt.title('Accuracy evolution')

In [ ]:
predicted_classes = model.predict(X_test)

In [ ]:
L = 5
W = 5

fig, axes = plt.subplots(L, W, figsize = (12,12))
axes = axes.ravel()

for i in np.arange(0, L * W):  
    axes[i].imshow(X_test[i].reshape(28,28))
    axes[i].set_title(f"PC = {Dev_list[np.argmax(predicted_classes[i])]}\n TC = {Dev_list[Y_test[i]]}")
    axes[i].axis('off')
plt.subplots_adjust(wspace=0.5)

In [ ]:
TC